In [ ]:
# cleaning...%load_ext autoreload
# cleaning...%autoreload 2

In [ ]:
import numpy as np
import scipy as sci
import statsmodels as stat

import kraft

In [ ]:
def objective_function_test(parameters):

    a, b = parameters

    return float(a + b)


sci.optimize.minimize(objective_function_test, (8, 8), bounds=((0, None), (0, None)))

In [ ]:
def objective_function(parameters, y_pdf, model, x):

    degree_of_freedom, shape, location, scale = parameters

    return (
        (y_pdf - model.pdf(x, degree_of_freedom, shape, loc=location, scale=scale))
        ** 2
        ** 0.5
    ).sum()

In [ ]:
location = 0

scale = 1

y = np.random.normal(loc=location, scale=scale, size=128)

over = (y.max() - y.min()) * 0

x = np.linspace(y.min() - over, y.max() + over, num=y.size)

model = stat.sandbox.distributions.extras.ACSkewT_gen()

fit_result = model.fit(y, loc=location, scale=scale)

y_fit_pdf = model.pdf(
    x, fit_result[0], fit_result[1], loc=fit_result[2], scale=fit_result[3]
)

for method in (
    "Nelder-Mead",
    "Powell",
    "CG",
    #     "BFGS",
    #     "Newton-CG",
    #     "L-BFGS-B",
    #     "TNC",
    #     "COBYLA",
    #     "SLSQP",
    #     "trust-constr",
    #     "dogleg",
    #     "trust-ncg",
    #     "trust-exact",
    #     "trust-krylov",
):

    print(method)

    minimize_result = sci.optimize.minimize(
        objective_function,
        (1, 0, location, scale),
        args=(y, model, x),
        method=method,
        bounds=((0, None), (0, 24), (-8, 8), (0, None)),
    )

    kraft.plot_plotly_figure(
        {
            "layout": {"legend": {"orientation": "h"}},
            "data": [
                {
                    "type": "histogram",
                    "name": "Y Distribution",
                    "x": y,
                    "histnorm": "probability density",
                },
                {
                    "type": "scatter",
                    "name": "Y PDF (parameters from fit)",
                    "x": x,
                    "y": y_fit_pdf,
                },
                {
                    "type": "scatter",
                    "name": "Y PDF (parameters from minimize)",
                    "x": x,
                    "y": model.pdf(
                        x,
                        np.e ** minimize_result.x[0],
                        minimize_result.x[1],
                        loc=minimize_result.x[2],
                        scale=minimize_result.x[3],
                    ),
                },
            ],
        },
        None,
    )